In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, lit 

import pandas as pd


import os
from pyspark.sql import Window 

from preprocessing import get_player_movement_data, cleansing_score_data, create_posessions, create_value_x_posession


# Create a Spark session
spark = (
    SparkSession
    .builder 
    .appName("Preprocessing") 
    .config("spark.driver.memory", "2g") 
    .getOrCreate()
)



your 131072x1 screen size is bogus. expect trouble
24/06/02 14:55:32 WARN Utils: Your hostname, MyLaptop resolves to a loopback address: 127.0.1.1; using 172.28.44.164 instead (on interface eth0)
24/06/02 14:55:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/02 14:55:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
score_data = spark.read.options(header="true").csv(".temp/score_data_final")


In [3]:
DICT_MONTHS = {
    '01': 'January', 
    '02': 'February', 
    '03': 'March', 
    '04': 'April', 
    '05': 'May', 
    '06': 'June', 
    '07': 'July', 
    '08': 'August', 
    '09': 'September', 
    '10': 'October', 
    '11': 'November', 
    '12': 'December', 
}

In [4]:
# pd.read_parquet(".temp/score_data_final", 
#                     filters=[("Date", "==", f"{DICT_MONTHS[info_game[0]]} {str(int(info_game[1]))} {info_game[2]}")])['HomeTeam'].value_counts()

In [6]:
for root, dirs, files in os.walk("../data"):
    for file in files:
        file_path = os.path.join(root, file)
        if file.endswith('json'):
            ruta_folder = file_path.split("/")[:-1]
            if (len(ruta_folder[-1]) == 21) & (ruta_folder[-1] not in os.listdir(f"../data/parquet/")):
                ruta_folder_path = "/".join(ruta_folder)
                info_game = ruta_folder[-1].split(".")

                events_data = spark.read.option("multiLine", "true").json(ruta_folder_path).select(F.explode("events").alias("event"))

                moments_data = events_data.select(F.explode("event.moments").alias("moments"))
                moments_data.write.mode("overwrite").parquet(".temp/temp")
                moments_data = spark.read.parquet(".temp/temp")

                moments_data = moments_data.dropDuplicates()
                moments_data = moments_data.filter((F.monotonically_increasing_id() % 5) == 0)

                player_movement_data = get_player_movement_data(moments_data)

                player_movement_data.write.mode("overwrite").parquet(".temp/player_movement_data")
                player_movement_data = spark.read.parquet(".temp/player_movement_data")       

                if info_game[-1] == 'CHA': info_game[-1] = 'CHO'
                if info_game[-1] == 'BKN': info_game[-1] = 'BRK'
                if info_game[-1] == 'PHX': info_game[-1] = 'PHO'
                
                score_data = pd.read_parquet(".temp/score_data_final", 
                    filters=[("Date", "==", f"{DICT_MONTHS[info_game[0]]} {str(int(info_game[1]))} {info_game[2]}"),
                            ("HomeTeam", "==", f"{info_game[-1]}")]) 
                
                score_data = cleansing_score_data(score_data)
                score_data = create_posessions(score_data)
                score_data = create_value_x_posession(score_data)
                

                score_data.to_parquet(".data/posessions_data.parquet")
                score_data = spark.read.parquet(".data/posessions_data.parquet")

                cond = (
                    (player_movement_data['quarter'] == score_data['quarter_score']) & 
                    (player_movement_data['game_clock'] < score_data['secleft']) & 
                    (player_movement_data['game_clock'] > score_data['secleft_end']) 
                )

                player_movement_w_scores = player_movement_data.join(F.broadcast(score_data), on=cond, how="left")

                player_movement_w_scores = player_movement_w_scores.drop(*["quarter_score", "secleft", "secleft_end"])
                player_movement_drop_duplicates = player_movement_w_scores.dropDuplicates()

                player_movement_drop_duplicates.write.mode("overwrite").parquet(f"../data/parquet/{ruta_folder[-1]}")

/mnt/c/MDS/Q2/SDM/project/NBA-Player-Movements/analysis/preprocessing.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_data['Secleft_end'] = score_data['secleft'].shift(-1)
/mnt/c/MDS/Q2/SDM/project/NBA-Player-Movements/analysis/preprocessing.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_data['y_away'] = score_data['awayscore'].shift(-1) - score_data['awayscore']
/mnt/c/MDS/Q2/SDM/project/NBA-Player-Movements/analysis/preprocessing.py:108: SettingWithCopyWarning: 
A value is trying t

KeyboardInterrupt: 

In [6]:

home_team_info = events_data.select("event.home")
away_team_info = events_data.select("event.visitor")

players_home = (
    home_team_info
    .select(F.explode("home.players").alias("PlayersHome"))
    .selectExpr("PlayersHome.*")
)

players_visitor= (
    away_team_info
    .select(F.explode("visitor.players").alias("PlayersVisitor"))
    .selectExpr("PlayersVisitor.*")
)


In [7]:

players_names = []
players_x_coord = []
players_y_coord = []
players_team = []

for player in range(0,11):
    flattened_player = flatten_players_movement_data(player=player)
    if player == 0: player_name = 'ball' 
    else: player_name = player

    players_names.append(flattened_player[1].alias(f"player{player_name}_name"))
    players_x_coord.append(flattened_player[2].cast("float").alias(f"player{player_name}_x_coord"))
    players_y_coord.append(flattened_player[3].cast("float").alias(f"player{player_name}_y_coord"))
    players_team.append(flattened_player[0].alias(f"player{player_name}_team"))


player_movement_data = (
moments_data
.withColumn("playerposition",col("moments")[5])
.withColumn("splitted",F.split("playerposition", r"\],\["))
.select(
    col("moments")[0].cast("integer").alias("quarter"),
    col("moments")[2].cast("float").alias("game_clock"),
    col("moments")[3].cast("float").alias("shot_clock"),
    *players_names, 
    *players_x_coord, 
    *players_y_coord,
    *players_team
    )
)

In [8]:
home_team_id = home_team_info.select("home.teamid")
visitor_team_id = away_team_info.select("visitor.teamid")

In [157]:
score_data = spark.read.parquet(".temp/score_data")

In [158]:
score_data_pd = (
    score_data
    .filter((col("date") == lit("January 14 2016")) & (col("HomeTeam") == 'UTA'))
    .toPandas()
)    

In [29]:
pd.read_parquet(".temp/score_data_final", 
                filters=[("Date", "==", f"{DICT_MONTHS['01']} 14 2016"),
                         ("HomeTeam", "==", "UTA")])

,URL,GameType,Location,Date,Time,WinningTeam,quarter_score,secleft,AwayTeam,AwayPlay,...,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss
0,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,1,720,SAC,Jump ball: W. Cauley-Stein vs. R. Gobert (R. G...,...,None,None,None,None,None,None,None,W. Cauley-Stein - caulewi01,R. Gobert - goberru01,R. Gay - gayru01
1,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,1,703,SAC,B. McLemore makes 2-pt jump shot from 5 ft,...,None,None,None,None,None,None,None,None,None,None
2,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,1,679,SAC,None,...,None,None,None,None,None,None,None,None,None,None
3,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,1,674,SAC,Shooting foul by R. Hood (drawn by D. Cousins),...,None,None,None,None,None,None,None,None,None,None
4,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,1,674,SAC,D. Cousins makes free throw 1 of 2,...,1 of 2,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,4,1,SAC,R. Gay makes 2-pt jump shot from 17 ft,...,None,None,None,None,None,None,None,None,None,None
503,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,4,0,SAC,None,...,None,None,None,None,None,None,None,None,None,None
504,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,4,0,SAC,None,...,None,None,None,None,None,None,None,None,None,None
505,/boxscores/201601140UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 14 2016,9:00 PM,SAC,4,0,SAC,End of 4th quarter,...,None,None,None,None,None,None,None,None,None,None
